## Trying Whisper Without N-best outputs

**Install whisper from git**

Make sure visibility is public

In [ ]:
!pip install git+https://github.com/johnlohjy/whisper.git@Nbest_transcriptions

  Cloning https://github.com/johnlohjy/whisper.git (to revision Nbest_transcriptions) to /tmp/pip-req-build-62zb6hut
  Running command git clone --filter=blob:none --quiet https://github.com/johnlohjy/whisper.git /tmp/pip-req-build-62zb6hut
  Running command git checkout -b Nbest_transcriptions --track origin/Nbest_transcriptions
  Switched to a new branch 'Nbest_transcriptions'
  Branch 'Nbest_transcriptions' set up to track remote branch 'Nbest_transcriptions' from 'origin'.
  Resolved https://github.com/johnlohjy/whisper.git to commit 7b1c525cb96e5d846973458fa4c4bda74bf79844
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=809412 sha256=6e7bbf6d17eb0248bc

**Load the Datasets from HuggingFace**

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 471.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
#Login because of gated access
#Token is saved to cache and available across all huggingface_hub components
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load the dataset
# gigapseech takes about 5mins
from datasets import load_dataset
ds_train = load_dataset("speechcolab/gigaspeech", "xs")

gigaspeech.py:   0%|          | 0.00/16.6k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

The repository for speechcolab/gigaspeech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/speechcolab/gigaspeech.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


data/xs_n_archives.txt:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

data/test_n_archives.txt:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

test_chunks_0001_metadata.csv:   0%|          | 0.00/2.66M [00:00<?, ?B/s]

test_chunks_0002_metadata.csv:   0%|          | 0.00/1.49M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# Seeing the dstrain dataset
ds_train

DatasetDict({
    train: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 9389
    })
    validation: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 6750
    })
    test: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 25619
    })
})

**Example 1**

In [ ]:
# Load the first example on the fly
ds_train["train"][0]

{'segment_id': 'YOU0000000315_S0000660',
 'speaker': 'N/A',
 'text': "AS THEY'RE LEAVING <COMMA> CAN KASH PULL ZAHRA ASIDE REALLY QUICKLY <QUESTIONMARK>",
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/77906a935e2abb2d9cf4f707fdb5c12b788afa9bc712c39fef49bf186c19af8f/xs_chunks_0000/YOU0000000315_S0000660.wav',
  'array': array([0.0005188 , 0.00085449, 0.00012207, ..., 0.00125122, 0.00076294,
         0.00036621]),
  'sampling_rate': 16000},
 'begin_time': 2941.889892578125,
 'end_time': 2945.070068359375,
 'audio_id': 'YOU0000000315',
 'title': 'Return to Vasselheim | Critical Role: VOX MACHINA | Episode 43',
 'url': 'https://www.youtube.com/watch?v=zr2n1fLVasU',
 'source': 2,
 'category': 24,
 'original_full_path': 'audio/youtube/P0004/YOU0000000315.opus'}

In [21]:
import whisper

model = whisper.load_model("base")
result = model.transcribe(ds_train["train"][0]["audio"]["path"], beam_size= 10)

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [22]:
result

[{'text': " As you're leaving, can cache pulls RSI really quickly?",
  'segments': [{'seek': 0,
    'start': 0.0,
    'end': 3.12,
    'text': " As you're leaving, can cache pulls RSI really quickly?",
    'tokens': [50364,
     1018,
     291,
     434,
     5012,
     11,
     393,
     19459,
     16982,
     497,
     20262,
     534,
     2661,
     30,
     50520],
    'temperature': 0.0,
    'avg_logprob': -0.3996254801750183,
    'compression_ratio': 0.9,
    'no_speech_prob': 0.013363807462155819}],
  'language': 'en'},
 {'text': " As you're leaving, can cache pulls RSI really quickly?",
  'segments': [{'seek': 0,
    'start': 0.0,
    'end': 3.04,
    'text': " As you're leaving, can cache pulls RSI really quickly?",
    'tokens': [50364,
     1018,
     291,
     434,
     5012,
     11,
     393,
     19459,
     16982,
     497,
     20262,
     534,
     2661,
     30,
     50516],
    'temperature': 0.0,
    'avg_logprob': -0.3996254801750183,
    'compression_ratio': 0.

In [23]:
def retrieve_n_best_transcriptions(result):
  n_best = []
  for d in result:
    n_best.append(d['text'])
  return n_best

n_best = retrieve_n_best_transcriptions(result)

for text in n_best:
  print(text)

 As you're leaving, can cache pulls RSI really quickly?
 As you're leaving, can cache pulls RSI really quickly?
 As you're leaving, can cache pulls RSI really quickly?
 As you're leaving, can cache pulls RSI really quickly?
 As you're leaving, can cache pulls RSI really quickly?
 As you're leaving, can cache pulls RSI really quickly?
 As your leaving can cache pulls RSI really quickly.
 As your leaving can cache pulls RSI really quickly.
 As your leaving can cache pulls RSI really quickly.
 As your leaving can cache pulls RSI really quickly.


**Example 2**

In [24]:
# Load the second example on the fly
ds_train["train"][3]

{'segment_id': 'YOU0000000299_S0000629',
 'speaker': 'N/A',
 'text': 'TO HELP SCREEN READER USERS IN THE MIDST OF DIV SOUP AND SPAN SALAD <PERIOD>',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/77906a935e2abb2d9cf4f707fdb5c12b788afa9bc712c39fef49bf186c19af8f/xs_chunks_0000/YOU0000000299_S0000629.wav',
  'array': array([0.00021362, 0.00036621, 0.00054932, ..., 0.00030518, 0.00030518,
         0.00024414]),
  'sampling_rate': 16000},
 'begin_time': 2553.739990234375,
 'end_time': 2557.77001953125,
 'audio_id': 'YOU0000000299',
 'title': 'Making Accessible Web Apps Using HTML5 and ChromeVox',
 'url': 'https://www.youtube.com/watch?v=x18vEEfpK3g',
 'source': 2,
 'category': 15,
 'original_full_path': 'audio/youtube/P0003/YOU0000000299.opus'}

In [25]:
result = model.transcribe(ds_train["train"][3]["audio"]["path"], beam_size= 10)

In [26]:
n_best = retrieve_n_best_transcriptions(result)

for text in n_best:
  print(text)

 to help screen reader users in the midst of div soup and span salad.
 to help screen reader users in the midst of VivSoup and SpanSallet.
 to help screen reader users in the midst of BivSoup and SpanSallet.
 to help screen reader users in the midst of DIVSoup and SpANS salad.
 to help screen reader users in the midst of DIVSoup and SpanSallet.
 to help screen reader users in the midst of VivSoup and SpanSallet.
 to help screen reader users in the midst of BivSoup and SpanSallet.
 to help screen reader users in the midst of DIVSoup and SpAN salad.
 to help screen reader users in the midst of VivSoup and SpanSallet.
 to help screen user users in the midst of DIVSoup and SpANS salad.
